###### CSCE 320 :: Principles of Data Science ::Texas A&M University :: Fall 2020


# Homework 4: Clustering

### 100 points [7% of your final grade]
### Due: October 21 by 11:59pm

*Goals of this homework:* There are two main objectives of this homework: (i) implement k-means from scratch, (ii)  apply k-means on a toy dataset (the iris data), and (iii) explore the clustering of text data.

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw4.ipynb`. For example, if your UIN is `123456789`, then your homework submission would be something like `123456789_hw4.ipynb`. Submit this notebook via Canvas (looking for homework 4 under the assignment section; detailed instructions can be found [here](https://community.canvaslms.com/t5/Student-Guide/How-do-I-upload-a-file-as-an-assignment-submission-in-Canvas/ta-p/274)). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after October 24 at 11:59pm.

### Collaboration declarations:

You should add in all of your collaboration declarations here. 

* Collaboration Declaration 1
* Collaboration Declaration 2
* ...

*Recall our homework collaboration policy: Your homework is yours alone and you are expected to complete each homework independently. Your solution should be written by you without the direct aid or help of anyone else. However, we believe that collaboration and team work are important for facilitating learning, so we encourage you to discuss problems and general problem approaches (but not actual solutions) with your classmates. If you do have a chat with another student about a homework problem, you must inform us by writing a note on your homework submission (e.g., Bob pointed me to the relevant section for problem 3). The basic rule is that no student should explicitly share a solution with another student (and thereby circumvent the basic learning process), but it is okay to share general approaches, directions, and so on. If you feel like you have an issue that needs clarification, feel free to contact either me or the TA.*

## (30 points) Part 1: Implement k-means
  
For the first part of this assignment, you are going to implement k-means from (almost) scratch, which means you can only use [The Python Standard Library](https://docs.python.org/3/library/). 
We have predefined some helper functions for the ease of implementation and grading. However, you should always keep the main function's structure in mind while implementing those helper functions.

In [87]:
# Within the scope of The Python Standard Library, import all packages you need for the implementation of k-means
import random
from math import *
from collections import namedtuple
from copy import copy
from decimal import *

**Helper Function 1:** Initialize Centroids  
To start with, you need to pick k initial centroids from the data.

In [305]:
def initializeCentroids(datapoints, k):
    centroids = random.sample(datapoints,k)
    return centroids

In [ ]:
# optional: you may fake some simple data to test your implementation of initializeCentroids here
### your code here ###

**Helper Function 2:** Termination Condition  
Next, you should determine the termination condition for the loop. This function should return True or False if k-means is done. K-means terminates either because it has run a maximum number of iterations OR the centroids stop changing (your implementation should handle both cases).

In [61]:
def termination(oldCentroids, centroids, iterations, max_iterations):
    count = 0
    if(max_iterations == iterations):
        return True
    if((centroids == None) or (oldCentroids == None)):
        return False
    if(len(centroids) == len(oldCentroids)):
        for i in range(len(oldCentroids)):
            if(centroids[i] == oldCentroids[i]):
                count += 1
            if(count == len(oldCentroids)):
                return True
    return False

In [ ]:
# optional: you may fake some simple data to test your implementation of termination here
### your code here ###

**Helper Function 3:** Assign Cluster  
Now, you can place each datapoint in the cluster whose current centroid is nearest.

In [308]:
def assignCluster(datapoints, centroids):
    clusters = [[],[],[]]
    dist = []
    p1 = 0
    oncce = 0
    dim = len(datapoints[0])
    for i in datapoints:
        for j in centroids:
            for d in range(dim):
                if(isinstance(j[d],list)):
                    once = j[d][0]
                else:
                    once = j[d]
                p1 += (i[d]-once)**2
            p1 = sqrt(p1)
            dist.append(p1)
        if((dist[0] < dist[1]) and (dist[0] < dist[2])):
            minimum = dist[0]
        elif((dist[1] < dist[2]) and (dist[1] < dist[0])):
            minimum = dist[1]
        else:
            minimum = dist[2]
        clusters[dist.index(minimum)].append(i)
        dist=[]
        p1 = 0
    return clusters #Clusters a list of a list

In [271]:
# optional: you may fake some simple data to test your implementation of assignCluster here

**Helper Function 4:** Calculate Centroids  
Finally, the last helper function that calculates the new centroid of each cluster.

In [319]:
def calculateCentroids(clusters):
    centroids = []
    count = 0
    total = 0
    for a in clusters:
        size = len(a)
        divis = len(a)
        for b in range(0,4):
            sum1 = 0
            for c in a:
                sum1 += c[b]
            if(b == 0):
                centroids.append([sum1/(divis*1.0)])
            else:
                centroids[count].append([sum1/(divis*1.0)])
        count +=1
    return centroids

In [317]:
# optional: you may fake some simple data to test your implementation of calculateCentroids here
### your code here ###

**Main Function**  
It's time to wrap everything up! This function takes datapoints, k, and max_iteration as input; and it returns **a list of length k in which each element is a cluster of datapoints** and **a list of k centroids**. Also, you may want to **preserve the corresponding order of these two lists for the ease of implementation in part 2**.

In [281]:
def kmeans(datapoints, k, max_iterations):
    # initialization
    datapoints = datapoints.values.tolist()
    iterations=0
    dim = len(datapoints[0])
    oldCentroids = None
    centroids = initializeCentroids(datapoints, k)
    # Run the main k-means algorithm
    while not termination(oldCentroids, centroids, iterations, max_iterations):
        oldCentroids = centroids
        # Update recordings
        iterations += 1
        
        # Assign clusters based on centroids
        clusters = assignCluster(datapoints, centroids)
        
        # Update centroids
        centroids = calculateCentroids(clusters)

    return clusters, centroids

## (30 points) Part 2: Clustering Irises
For part 2 of this assignment, let's apply the k-means clustering function for plant taxonomy. Specifically, we will try to cluster some irises based on their sepal's and petal's length and width.  

**For this part, you will need to:**
1. Apply the k-means clustering function you just implemented on the iris data with k=1,2,3,...,9,10;
2. Calculate the average distance to centroid for each k;
3. Plot the k vs. average distance to centroid graph to pick the best value of k.

In [275]:
import pandas as pd

datapoints = pd.read_csv('Iris.tsv',sep='\t')
print(datapoints.head())

   sepal_length  sepal_width   petal_length   petal_width
0           5.1          3.5            1.4           0.2
1           4.9          3.0            1.4           0.2
2           4.7          3.2            1.3           0.2
3           4.6          3.1            1.5           0.2
4           5.0          3.6            1.4           0.2


In [327]:
%matplotlib inline
import matplotlib.pyplot as plt
kmeans(datapoints,3,100)


([[[7.0, 3.2, 4.7, 1.4],
   [6.4, 3.2, 4.5, 1.5],
   [6.9, 3.1, 4.9, 1.5],
   [5.5, 2.3, 4.0, 1.3],
   [6.5, 2.8, 4.6, 1.5],
   [5.7, 2.8, 4.5, 1.3],
   [6.3, 3.3, 4.7, 1.6],
   [6.6, 2.9, 4.6, 1.3],
   [5.2, 2.7, 3.9, 1.4],
   [5.9, 3.0, 4.2, 1.5],
   [6.0, 2.2, 4.0, 1.0],
   [6.1, 2.9, 4.7, 1.4],
   [5.6, 2.9, 3.6, 1.3],
   [6.7, 3.1, 4.4, 1.4],
   [5.6, 3.0, 4.5, 1.5],
   [5.8, 2.7, 4.1, 1.0],
   [6.2, 2.2, 4.5, 1.5],
   [5.6, 2.5, 3.9, 1.1],
   [5.9, 3.2, 4.8, 1.8],
   [6.1, 2.8, 4.0, 1.3],
   [6.3, 2.5, 4.9, 1.5],
   [6.1, 2.8, 4.7, 1.2],
   [6.4, 2.9, 4.3, 1.3],
   [6.6, 3.0, 4.4, 1.4],
   [6.8, 2.8, 4.8, 1.4],
   [6.7, 3.0, 5.0, 1.7],
   [6.0, 2.9, 4.5, 1.5],
   [5.5, 2.4, 3.8, 1.1],
   [5.8, 2.7, 3.9, 1.2],
   [6.0, 2.7, 5.1, 1.6],
   [5.4, 3.0, 4.5, 1.5],
   [6.0, 3.4, 4.5, 1.6],
   [6.7, 3.1, 4.7, 1.5],
   [6.3, 2.3, 4.4, 1.3],
   [5.6, 3.0, 4.1, 1.3],
   [5.5, 2.5, 4.0, 1.3],
   [5.5, 2.6, 4.4, 1.2],
   [6.1, 3.0, 4.6, 1.4],
   [5.8, 2.6, 4.0, 1.2],
   [5.6, 2.7, 4.2, 1.3],


**What is the best value of k according to your graph?**  
``TODO``

## (40 points) Part 3: Clustering Text Data
For the 3rd part of this assignment, you will explore the clustering of text data. The good news: for this part, you can use scikit-learn!

Let's take a look at the data:

In [65]:
import pandas as pd

df = pd.read_csv('product_reviews.tsv',sep='\t')
print('Preview:')
print(df.head())
print('\nShape:')
print(df.shape)
print('\nValue Counts:')
print(df['product_type'].value_counts())

Preview:
      product_type                                     review_content
0  Washing_Machine  Electrolux's Editors' Choice award-winning $1,...
1  Washing_Machine  GE's $699 GTW685BSLWS top-load washing machine...
2  Washing_Machine  Samsung's $1,899 WV9900 FlexWash is actually t...
3  Washing_Machine  The $650 GE GTW485ASJWS might be one of the si...
4  Washing_Machine  This $1,200 top-load washing machine seemingly...

Shape:
(7758, 2)

Value Counts:
Phone              1212
Laptop             1069
Camera              834
TV                  794
Headphone           653
Speaker             551
Printer             359
Networking          355
Tablet              282
Monitor             258
Blu-Ray_Player      229
Desktop             216
Video_Camera        206
AV_Receiver         149
Small_Appliance     114
Wearable            107
Media_Streamers      91
Oven                 53
Refrigerator         49
Coffee_Maker         49
Vacuum_Cleaner       45
Washing_Machine      38
Dryer     

The dataset contains 7758 product review articles of 25 different types of products. **The goal for this part is to derive a fixed smaller amount of supergroups (say 5~10 clusters; pick your favorite number!) with different clustering algorithms based on the content of reviews of 25 different types of products.**  
Before feeding the clustering functions, you need to transform the data a little bit so that the clustering functions can digest the review articles.  

First, you should represent each review as a vector, where element i of the vector corresponds to the count of word i in that product review. This is the classic [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Besides that, you also need to use lower case text, do tokenization, remove punctuations, remove [stop words](https://en.wikipedia.org/wiki/Stop_word), use [n-gram](https://en.wikipedia.org/wiki/N-gram) tokens, etc. for better results.   
Now, let's implement all those things.

In [ ]:
### don't actually add any code here ... keep reading ... ###

**<span style="color:red">Wait a minute!</span> sklearn can do it for your in one line : )**  
Take a look at [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and apply it to our corpus below. Alternatively, you can opt to implement all those things in the block above with **NO** extra credits :P  
**CountVectorizer has a lot of parameters; set them wisely.**

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = df.loc[:, 'review_content'].to_list()
Vectorizer = CountVectorizer(stop_words='english',ngram_range=(1,2),strip_accents='unicode',max_features=20)
M = Vectorizer.fit_transform(corpus)
print(Vectorizer.get_feature_names())

['battery', 'better', 'camera', 'display', 'features', 'good', 'inch', 'just', 'like', 'll', 'mode', 'phone', 'quality', 'samsung', 'screen', 'sound', 'time', 'tv', 'use', 'video']


Rather than simply consider word counts as the representation of reviews (as we do in CountVectorizer), we can also consider alternative weightings of the words in our vector representation of each review. One of the classic approaches is [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf), which weighs a term in a document highly if it is popular within the document (term frequency) and also rare across all documents (inverse document frequency). Take a look at [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and apply it to our corpus below.

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
TfidVecto = TfidfVectorizer(strip_accents='unicode',stop_words='english',max_features=20)
X = TfidVecto.fit_transform(corpus)
print(TfidVecto.get_feature_names())
M1 = TfidfTransformer(smooth_idf=True)
M1.fit(X)
print(M1.idf_)

['battery', 'better', 'camera', 'display', 'features', 'good', 'inch', 'just', 'like', 'll', 'mode', 'phone', 'quality', 'samsung', 'screen', 'sound', 'time', 'tv', 'use', 'video']
[1.76072335 1.44664159 1.99384481 1.71017495 1.41745838 1.29756882
 1.51413909 1.28515059 1.17091629 1.41296838 1.79666208 2.02748225
 1.43263856 2.29885347 1.47878027 1.88789255 1.45635827 2.31456434
 1.26950401 1.61615273]


Now, it's time to do clustering! In the following sections, apply k-means and another clustering algorithm of your choice (within the scope of [sklearn.cluster](https://scikit-learn.org/stable/modules/clustering.html)) to the product review corpus. Try both count vectors and tf-idf vectors, and record:
- Cluster Composition by Count: for each cluster, count the amount of reviews by product_type;
- Cluster Composition by Percentage: for each product_type, calculate its reviews' distribution (in percentage) across clusters.

Recall that you need to pick a number between 5 and 10 as your target amount of supergroups.

In [ ]:
from sklearn import cluster
#### K-Means - Count
print("\n\nK-Means clustering by count")
kmeans = cluster.KMeans(n_clusters=7)
kmeans.fit_predict(M)
labels = kmeans.labels_
TypeCount = pd.DataFrame(labels,index=df['product_type'],columns=['Cluster'])
Product_count = TypeCount.groupby(['product_type','Cluster']).size()
print("Product Count for each cluster",Product_count)
TypeCount['kmean'] = kmeans.labels_
print("Total values in each cluster",TypeCount['kmean'].value_counts())
#### K-Means - TF-IDF
print("\n\nK-Means clustering by tf-idf")
kmeans.fit_predict(X)
labels = kmeans.labels_
TypeCount_TF = pd.DataFrame(labels,index=df['product_type'],columns=['Cluster'])
Product_count = TypeCount_TF.groupby(['product_type','Cluster']).size()
print("Product Count for each cluster",Product_count)
TypeCount_TF['kmean'] = kmeans.labels_
print("Total values in each cluster",TypeCount_TF['kmean'].value_counts())
##### Spectral by Count
print("\n\nSpectral Clustering by count")
SpClust = cluster.SpectralClustering(n_clusters=7)
SpClust.fit_predict(M)
labels = SpClust.labels_
TypeCount_SP_Cnt = pd.DataFrame(labels,index=df['product_type'],columns=['Cluster'])
Product_count_SP_Cnt = TypeCount_SP_Cnt.groupby(['product_type','Cluster']).size()
print("Product Count for each cluster", Product_count_SP_Cnt)
TypeCount_SP_Cnt['Spectral'] = SpClust.labels_
print("Total values in each cluster",TypeCount_SP_Cnt['Spectral'].value_counts())
##### Spectral by TF-IDF
print("\n\nSpectral Clustering by tf-idf")
SpClust = cluster.SpectralClustering(n_clusters=7)
SpClust.fit_predict(X)
labels = SpClust.labels_
TypeCount_SP = pd.DataFrame(labels,index=df['product_type'],columns=['Cluster'])
Product_count_SP = TypeCount_SP.groupby(['product_type','Cluster']).size()
print("Product Count for each cluster", Product_count_SP)
TypeCount_SP['Spectral'] = SpClust.labels_
print("Total values in each cluster",TypeCount_SP['Spectral'].value_counts())



K-Means clustering by count
Product Count for each cluster product_type     Cluster
AV_Receiver      0          55
                 1          73
                 2           1
                 6          20
Blu-Ray_Player   0           3
                            ..
Video_Camera     6          35
Washing_Machine  1          38
Wearable         1          72
                 4          22
                 6          13
Length: 84, dtype: int64
Total values in each cluster 1    4470
6    1006
3     696
4     666
0     412
2     406
5     102
Name: kmean, dtype: int64


K-Means clustering by tf-idf
Product Count for each cluster product_type  Cluster
AV_Receiver   0          109
              1            1
              3            6
              4           26
              5            6
                        ... 
Wearable      1           15
              3            1
              4           47
              5           13
              6           31
Length: 117, dtype: 

C:\Users\olivi\AnacondaDS\envs\cs320\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


You may use the following table or use plots to illustrate your experiment results:  

|  Model  | Vector | Cluster Composition by Count | Cluster Composition by Percentage |
|:-------:|:------:|:----------------------------:|:---------------------------------:|
| k-means |  count |            Max=1023 at Cluster 5            |               `TODO`              |
| k-means | tf-idf |            Max=1417 at Cluster 4            |               `TODO`              |
|  Ward hierarchical clustering |  count |      Max=4480 at Cluster 1         |       `TODO`              |
|  Ward hierarchical clustering | tf-idf |            Max=1841 at Cluster 1           |               `TODO`              |

Analyze your experiment results and add a brief discussion here on:
- which setting yields better clusters (supergroups)?
- what're your justifications for your answer of the above question?
- any other observations?

`TODO`